In [1]:
from pathlib import Path
import torch, torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
device="cuda"

In [2]:
data_dir = Path('./data/tiny-imagenet-200')
data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.4805, 0.4483, 0.3978), (0.263, 0.257, 0.267)),
])

In [3]:
valid_set = torchvision.datasets.ImageFolder(data_dir / 'val', data_transforms)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=8,
                                           shuffle=True, num_workers=4, pin_memory=True)
loss = CrossEntropyLoss()

In [4]:
model_path = 'outputs/best_models/resnet50/resnet50_adv_val_acc=0.7623.pt'
model = torchvision.models.resnet50()
model.fc = nn.Linear(2048, 200)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
print('model loaded')

model loaded


In [5]:
def valid(model, dataset, dataloader, loss, device):
    running_loss = 0.0
    right = 0
    total = 0
    model.eval()
    for i, data in tqdm(enumerate(dataloader), total=len(dataset) / dataloader.batch_size):
        data, y = data
        data = data.to(device)
        y = y.to(device)
        ans = model.forward(data)
        los = loss(ans, y)
        running_loss += los.item()
        right += torch.sum(torch.eq(torch.argmax(ans, dim=1), y))
        total += y.shape[0]
    return running_loss, running_loss / len(dataloader.dataset), right/total

In [6]:
valid_loss, avg_valid, valid_acc = valid(model, valid_set, valid_loader, loss, device)
print('accuracy:', valid_acc.item())

100%|████████████████████████████████████████████████████████████████████████████| 1250/1250.0 [00:46<00:00, 26.60it/s]

accuracy: 0.7601000070571899
